# Test AI Query

## Test Ideas:

* operator performance
* flag which conversation are against policy
* how they could have been improved (you forgot to suggest x/y/z to the customer), and then providing customized recommendation to the agents for example?



In [0]:
USE CATALOG fins_genai;
USE SCHEMA call_center;

In [0]:
select 
  *
from transcripts_ai_query_test
limit 10;

call_duration_in_sec call_timestamp operator_id phone_number transcript 411 2024-07-18T10:33:55.000Z 7007 (819)-921-1828 all calls are recorded for training and monitoring purposes. My car's audio system was stolen and I need to file a claim., Oh, so you're another one of those people who thinks they can just leave their car unlocked and not have anything stolen, huh? Well, let me tell you, that's just not how the world works. You gotta take responsibility for your own property, pal. Now, I know you're here to file a claim for your precious audio system, but let me tell you, it's not gonna be easy. You're gonna have to jump through some hoops, provide all sorts of documentation, and maybe, just maybe, we'll cut you a check. But don't get your hopes up too high, pal. So, what kind of audio system was it? One of those fancy schmancy ones with all the bells and whistles? Ha! Like that's gonna make a difference. You're still gonna have to pay your deductible, and don't even get me started on the paperwork. But hey, I'm a nice guy, so I'll give you a break. I'll make sure to put a rush on your claim, even though it's not like you're gonna get a brand new audio system out of the deal. You'll be lucky if we give you a few bucks for a new radio. So, what do you say? Ready to dance with the devil and file that claim? Or are you gonna take your chances and hope your car doesn't get broken into again? The choice is yours, pal. But either way, don't come crying to me when things don't go your way., Sure, I'd be happy to help you file a claim for your stolen audio system. However, I must inform you that our agency's policy requires a thorough investigation before any claims can be approved. Additionally, our team of experts will need to review the circumstances surrounding the theft to ensure that all necessary protocols were followed. I understand that this process can be frustrating, but please rest assured that we are committed to providing the highest level of service to our clients. We will do everything in our power to make sure that your claim is processed as efficiently and effectively as possible. Regarding the audio system, I'm afraid that we cannot guarantee a brand-new replacement. However, we will do our best to provide a fair settlement based on the value of the stolen item. Please keep in mind that the settlement amount may be affected by factors such as depreciation and the age of the item. I hope this information helps clarify things for you. If you have any further questions or concerns, please don't hesitate to reach out. We're here to help in any way we can., I understand that you have a process to follow, but I'm concerned that it's going to take a long time to get my claim approved. Can you give me an idea of how long the investigation and review process typically takes? Also, I'm not sure if I have all the documentation that you need. Can you provide me with a list of what I'll need to provide in order to file the claim? I want to make sure that I have everything ready and in order to avoid any delays. Lastly, I'm not sure if I understand how the settlement amount is determined. Can you explain to me how you calculate the value of the stolen item and how depreciation affects the settlement amount? I appreciate your help and I'm willing to work with you to make sure that everything is in order, but I want to make sure that I'm well informed throughout the process., Sure, I'd be happy to help you file a claim for your stolen audio system. However, I must inform you that our agency's policy requires a thorough investigation before any claims can be approved. Additionally, our team of experts will need to review the circumstances surrounding the theft to ensure that all necessary protocols were followed. I understand that this process can be frustrating, but please rest assured that we are committed to providing the highest level of service to our clients. We will do everything in our power to make sure that your claim i

# Compliance

In [0]:
select * from call_center_guidelines

guidelines
"1. Inform customer that 'all calls are recorded for training and monitoring purposes'. 2. Greet the customer with a friendly and professional tone. 3. Verify customer information 4. provide a clear explanation of the resolution. 5. end the call by inform customer 'if you have more questions, you can always call us at 1-800-123-4567'"


In [0]:
with compliance as (
  select
    case when call_duration_in_sec > 480 then 'exceed_time_limit' else 'within_time_limit' end as time_limit,
    ai_query(
      "databricks-claude-3-7-sonnet",
      concat("Using the following 5 guidelines:", (select guidelines from call_center_guidelines),
            "to identify the violations from the customer service operators based on the following transrcipt:",
            "<transcript>", transcript, "</transcript>\n",
            "each violation on a guideline count as 1 violation and max number of violations is 5",
            "each operator starts with 10 points, deduct 1 point for each violations",
            "give justifaction for each violation reference to the guidelines using bullet points 'e.g. - violate guideline #: '",
            "return in a json {'points': INT, 'justification': STRING} without any Markdown formatting", 
            "do not explain"
            ),
      returnType => 'STRUCT<points:INT, justification:STRING>'
      -- responseFormat => '{"type": "json_object"}'
      ) as compliance_score
  from transcripts_ai_query_test limit 10
),

compliance_json as (
select 
  from_json(compliance_score, 'STRUCT<points:INT, justification:STRING>') as compliance_score
from compliance
)

select
  compliance_score.points as points,
  compliance_score.justification as justification,
  case when lower(justification) like '%guideline #1%' then 1 else 0 end as no_inform_compliance,
  case when lower(justification) like '%guideline #2%' then 1 else 0 end as unprofessionalism,
  case when lower(justification) like '%guideline #3%' then 1 else 0 end as no_identity_verification,
  case when lower(justification) like '%guideline #4%' then 1 else 0 end as no_detail_explaination,
  case when lower(justification) like '%guideline #5%' then 1 else 0 end as no_inform_company_contact
from compliance_json

points,justification,no_inform_compliance,unprofessionalism,no_identity_verification,no_detail_explaination,no_inform_company_contact
null,null,0,0,0,0,0
5,"- Violates guideline #2: The operator starts with ""Oh, great. Another claim"" and continues with a very unprofessional and hostile tone throughout their response, including phrases like ""let me tell you, pal, we're not made of money"" and ""don't think for a second that we're gonna replace your audio system"" - Violates guideline #3: The operator fails to properly verify customer information, saying things like ""don't bother giving me the VIN number, I already know it"" and ""I'll just give you a fake policy number"" - Violates guideline #4: Instead of providing a clear explanation of the resolution, the operator says things like ""don't think for a second that we're gonna give you a rental car"" and ""don't bother calling us back to check on the status of your claim"" - Violates guideline #5: The operator does not end the call by informing the customer they can call back with questions at 1-800-123-4567 - Violates guideline #1: While the transcript starts with ""all calls are recorded for training and monitoring purposes,"" this appears to be from the customer repeating it, not the operator informing the customer",1,1,1,1,1
9,"- Violate guideline #3: The operator did not fully verify the customer's information. While they confirmed the vehicle details (2020 Honda Civic) and policy number (ABC123456), they did not verify other essential customer information such as the customer's name, address, or contact details.",0,0,1,0,0
8,"- Violates guideline #3: The operator did not fully verify the customer's information. They mentioned ""The policy number associated with your account is HPI-"" but did not complete the policy number or verify other customer details like name, address, or date of birth. - Violates guideline #5: In the final response, the operator did not correctly end the call with the specific required statement. They said ""You can contact me anytime by replying to this message or starting a new conversation"" instead of using the exact required closing statement ""if you have more questions, you can always call us at 1-800-123-4567"".",0,0,1,0,1
5,"- Violates guideline #2: The operator does not greet the customer with a friendly and professional tone. Instead, they start with ""Oh, great. Another claim for damage caused by a freakin' tree branch"" and use an unprofessional tone throughout the conversation. - Violates guideline #2: The operator continues with unprofessional language and tone in subsequent responses, saying things like ""Well, well, well. Look who's getting all defensive on me. I like that. It's cute."" - Violates guideline #3: The operator does not verify any customer information throughout the entire conversation. - Violates guideline #4: The operator does not provide a clear explanation of the resolution. Instead, they say things like ""I'll just arbitrarily choose to approve or deny it, because that's how we do things around here."" - Violates guideline #2: The operator is consistently rude and accusatory, saying things like ""You're probably just trying to scam some cash out of us"" and ""I know you're just trying to scam us.""",0,1,1,1,0
7,"- Violates guideline #2: The operator did not greet the customer with a friendly and professional tone. Instead, they immediately responded with ""Sorry to hear that, but we're not interested in paying for your claim"" which is unprofessional and dismissive. - Violates guideline #3: The operator did not verify any customer information before discussing the claim details. - Violates guideline #4: The operator did not provide a clear explanation of the resolution. They simply dismissed the claim without proper explanation, saying ""We're not going to bother with the details of your claim"" and suggested the customer ""clean up the mess yourself.""",0,1,1,1,0
9,- Violates guideline #2: The operator did not

In [0]:
select
  *,
  case when call_duration_in_sec > 480 then 'exceed_time_limit' else 'within_time_limit' end as time_limit,
  ai_query(
    "databricks-claude-3-7-sonnet",
    concat("Using the following 5 guidelines:", (select guidelines from call_center_guidelines),
           "to identify the violations from the customer service operators based on the following transrcipt:",
           "<transcript>", transcript, "</transcript>\n",
           "each violation on a guideline count as 1 violation and max number of violations is 5"
           "each operator starts with 10 points, deduct 1 point for each violations"
           "return the final points in a json {'points': INT'}, do not explain"
           ""),
    returnType => 'STRUCT<points:INT>'
    -- responseFormat => '{"type": "json_object"}'
    ) as compliance_score,
  ai_query(
    "databricks-claude-3-7-sonnet",
    concat("Using the following guildlines:", (select guidelines from call_center_guidelines),
           "to identify the violations from the customer service operators based on the following transrcipt:",
           "<transcript>", transcript, "</transcript>\n", 
           "each violation on a guideline count as 1 violation and max number of violations is 5"
           "Give justifaction to each violation")
    ) as compliance_violations_justification
from transcripts_ai_query_test limit 10

call_duration_in_sec call_timestamp operator_id phone_number transcript time_limit compliance_score compliance_violations_justification 411 2024-07-18T10:33:55.000Z 7007 (819)-921-1828 all calls are recorded for training and monitoring purposes. My car's audio system was stolen and I need to file a claim., Oh, so you're another one of those people who thinks they can just leave their car unlocked and not have anything stolen, huh? Well, let me tell you, that's just not how the world works. You gotta take responsibility for your own property, pal. Now, I know you're here to file a claim for your precious audio system, but let me tell you, it's not gonna be easy. You're gonna have to jump through some hoops, provide all sorts of documentation, and maybe, just maybe, we'll cut you a check. But don't get your hopes up too high, pal. So, what kind of audio system was it? One of those fancy schmancy ones with all the bells and whistles? Ha! Like that's gonna make a difference. You're still gonna have to pay your deductible, and don't even get me started on the paperwork. But hey, I'm a nice guy, so I'll give you a break. I'll make sure to put a rush on your claim, even though it's not like you're gonna get a brand new audio system out of the deal. You'll be lucky if we give you a few bucks for a new radio. So, what do you say? Ready to dance with the devil and file that claim? Or are you gonna take your chances and hope your car doesn't get broken into again? The choice is yours, pal. But either way, don't come crying to me when things don't go your way., Sure, I'd be happy to help you file a claim for your stolen audio system. However, I must inform you that our agency's policy requires a thorough investigation before any claims can be approved. Additionally, our team of experts will need to review the circumstances surrounding the theft to ensure that all necessary protocols were followed. I understand that this process can be frustrating, but please rest assured that we are committed to providing the highest level of service to our clients. We will do everything in our power to make sure that your claim is processed as efficiently and effectively as possible. Regarding the audio system, I'm afraid that we cannot guarantee a brand-new replacement. However, we will do our best to provide a fair settlement based on the value of the stolen item. Please keep in mind that the settlement amount may be affected by factors such as depreciation and the age of the item. I hope this information helps clarify things for you. If you have any further questions or concerns, please don't hesitate to reach out. We're here to help in any way we can., I understand that you have a process to follow, but I'm concerned that it's going to take a long time to get my claim approved. Can you give me an idea of how long the investigation and review process typically takes? Also, I'm not sure if I have all the documentation that you need. Can you provide me with a list of what I'll need to provide in order to file the claim? I want to make sure that I have everything ready and in order to avoid any delays. Lastly, I'm not sure if I understand how the settlement amount is determined. Can you explain to me how you calculate the value of the stolen item and how depreciation affects the settlement amount? I appreciate your help and I'm willing to work with you to make sure that everything is in order, but I want to make sure that I'm well informed throughout the process., Sure, I'd be happy to help you file a claim for your stolen audio system. However, I must inform you that our agency's policy requires a thorough investigation before any claims can be approved. Additionally, our team of experts will need to review the circumstances surrounding the theft to ensure that all necessary protocols were followed. I understand that this process can be frustrating, but please rest assured that we are committed to providing the highest level of service to our clients. W


# Get internal document context from transcript

* Part 1 Automobile Liability Insurance
Bodily Injury Coverage Property Damage Coverage
* Part 2 Automobile Medical Payments
Coverage
* Part 3 Automobile Death Indemnity Insurance Coverage
* Part 4 Automobile Disability Income Protection Coverage
* Part 5 Uninsured Motorists Insurance Coverage
* Part 6 Protection Against Loss To The Auto

In [0]:
with customer_needs as (
  select
    *,
    ai_query(
      "databricks-claude-3-7-sonnet",
      concat("Analysis the following transcripts: ", "<transcript>", transcript, "<transcript>\n",
             "identify the main concern or question from the customer in one sentence")
      ) as customer_query
  from transcripts_ai_query_test limit 10
)

select
  cn.*,
  vs.section
from
  customer_needs cn
join
  lateral (
    select
      *
    from
      vector_search(
        index => "fins_genai.call_center.policy_docs_chunked_files_vs_index",
        query_text => cn.customer_query,
        num_results => 1
      )
  ) vs

call_duration_in_sec call_timestamp operator_id phone_number transcript customer_query section 411 2024-07-18T10:33:55.000Z 7007 (819)-921-1828 all calls are recorded for training and monitoring purposes. My car's audio system was stolen and I need to file a claim., Oh, so you're another one of those people who thinks they can just leave their car unlocked and not have anything stolen, huh? Well, let me tell you, that's just not how the world works. You gotta take responsibility for your own property, pal. Now, I know you're here to file a claim for your precious audio system, but let me tell you, it's not gonna be easy. You're gonna have to jump through some hoops, provide all sorts of documentation, and maybe, just maybe, we'll cut you a check. But don't get your hopes up too high, pal. So, what kind of audio system was it? One of those fancy schmancy ones with all the bells and whistles? Ha! Like that's gonna make a difference. You're still gonna have to pay your deductible, and don't even get me started on the paperwork. But hey, I'm a nice guy, so I'll give you a break. I'll make sure to put a rush on your claim, even though it's not like you're gonna get a brand new audio system out of the deal. You'll be lucky if we give you a few bucks for a new radio. So, what do you say? Ready to dance with the devil and file that claim? Or are you gonna take your chances and hope your car doesn't get broken into again? The choice is yours, pal. But either way, don't come crying to me when things don't go your way., Sure, I'd be happy to help you file a claim for your stolen audio system. However, I must inform you that our agency's policy requires a thorough investigation before any claims can be approved. Additionally, our team of experts will need to review the circumstances surrounding the theft to ensure that all necessary protocols were followed. I understand that this process can be frustrating, but please rest assured that we are committed to providing the highest level of service to our clients. We will do everything in our power to make sure that your claim is processed as efficiently and effectively as possible. Regarding the audio system, I'm afraid that we cannot guarantee a brand-new replacement. However, we will do our best to provide a fair settlement based on the value of the stolen item. Please keep in mind that the settlement amount may be affected by factors such as depreciation and the age of the item. I hope this information helps clarify things for you. If you have any further questions or concerns, please don't hesitate to reach out. We're here to help in any way we can., I understand that you have a process to follow, but I'm concerned that it's going to take a long time to get my claim approved. Can you give me an idea of how long the investigation and review process typically takes? Also, I'm not sure if I have all the documentation that you need. Can you provide me with a list of what I'll need to provide in order to file the claim? I want to make sure that I have everything ready and in order to avoid any delays. Lastly, I'm not sure if I understand how the settlement amount is determined. Can you explain to me how you calculate the value of the stolen item and how depreciation affects the settlement amount? I appreciate your help and I'm willing to work with you to make sure that everything is in order, but I want to make sure that I'm well informed throughout the process., Sure, I'd be happy to help you file a claim for your stolen audio system. However, I must inform you that our agency's policy requires a thorough investigation before any claims can be approved. Additionally, our team of experts will need to review the circumstances surrounding the theft to ensure that all necessary protocols were followed. I understand that this process can be frustrating, but please rest assured that we are committed to providing the highest level of service to our clients. We will do everything in our power to make

## Customer Intent and Context

* Structured output

In [0]:
select
    ai_query(
        "databricks-meta-llama-3-3-70b-instruct",
        "extract the customer intent (of either 'auto accident', 'home accident', 'motocycle', or 'policy change') and
        key context of the intent from the transcript" || transcript,
        responseFormat => 'STRUCT<result:STRUCT<intent:STRING, context:STRING>>'
    ) as key_info
from transcripts_ai_query_test limit 20

key_info
"{""intent"": ""auto accident"", ""context"": ""The customer's car audio system was stolen and they need to file a claim.""}"
"{""intent"": ""auto accident"", ""context"": ""car's audio system was stolen""}"
"{""intent"": ""policy change"", ""context"": ""home business insurance""}"
"{""intent"": ""policy change"", ""context"": ""home business insurance policy with both property and liability coverage""}"
"{""intent"": ""home accident"", ""context"": ""tree branch fell on car during windstorm""}"
"{""intent"": ""home accident"", ""context"": ""water damage in basement from heavy rainstorm, dispute over claim and premium increase""}"
"{""intent"": ""policy change"", ""context"": ""add a new car to existing auto insurance policy""}"
"{""intent"": ""policy change"", ""context"": ""increase coverage limit on homeowner""}"
"{""intent"": ""policy change"", ""context"": ""add coverage for personal belongings to renters insurance""}"
"{""intent"": ""policy change"", ""context"": ""update address on car insurance policy""}"


In [0]:
select
  misconduct,
  count(*)
from
(
select
    operator_id,
    ai_query(
        "databricks-meta-llama-3-3-70b-instruct",
        concat("You are an expert in insurance operation, flag any mistconduct: (in the catagory of 'rudeness', 'discrimination', 'data overreach')",
               " in the following transcript: <transcript>", transcript, "<transcript>\n",
               "Answer with the catagory of the mistconduct, do not explain. If there is no mistconduct, reply with 'none', and answer flag as a boolean"),
        responseFormat => 'STRUCT<misconduct:STRUCT<catagory:ARRAY<STRING>, flag:BOOLEAN>>'
      ) as misconduct
from transcripts_ai_query_test
) group by 1

misconduct,count(*)
"{""catagory"": [""rudeness""], ""flag"": true}",134
"{""catagory"": [""rudeness"", ""discrimination"", ""data overreach""], ""flag"": true}",12
"{""catagory"": [""data overreach""], ""flag"": true}",10
"{""catagory"": [""rudeness"", ""discrimination""], ""flag"": true}",43
"{""catagory"": [""discrimination"", ""rudeness"", ""data overreach""], ""flag"": true}",1
"{""catagory"": [""rudeness"", ""data overreach""], ""flag"": true}",5
"{""catagory"": [""none""], ""flag"": false}",191


In [0]:
select
  ai_query(
      "databricks-claude-3-7-sonnet",
      concat("extract the customer intent of either 'auto accident', 'home accident', 'motocycle', or 'policy related' 
      from the transcript:", transcript, "only return the intent, do not explain")
  --returnType => 'STRUCT<intent:STRING>',
  --failOnError => false
  ) as customer_intent
from transcripts_ai_query_test limit 10

customer_intent
auto accident
auto accident
policy related
policy related
auto accident
home accident
home accident
policy related
policy related
policy related


In [0]:
select
    operator_id,
    transcript,
    ai_query(
        "databricks-claude-3-7-sonnet",
        concat("You are an expert in customer relationship managemenanalysis the following transcript: <transcript>", 
                transcript, "<transcript>\n",
              "give a recommendation of the next best action from the following list: 'follow-up call', 'promotional email', 'automated email', 'apology email'"
               "Answer with the recommended action, do not explain. If no action is required, return 'none'.")
        responseFormat => 'STRUCT<misconduct:STRUCT<catagory:ARRAY<STRING>, flag:BOOLEAN>>'
      ) as next_best_action
from transcripts_ai_query_test

lower(next_best_action),count(*)
automated email,31
apology email,128
none,161
follow-up call,76


In [0]:
select
  ai_query(
    "databricks-meta-llama-3-3-70b-instruct",
    "Generate a few insurance call center guidelines for the phone operators"
  )

"ai_query(databricks-meta-llama-3-3-70b-instruct, Generate a few insurance call center guidelines for the phone operators, returnTypeDefaultValue)"
"Here are a few insurance call center guidelines for phone operators: **Introduction and Greeting** 1. **Answer calls promptly**: Answer calls within 3 rings, and greet the customer with a friendly and professional tone. 2. **Identify yourself and the company**: Clearly state your name, the insurance company's name, and the purpose of the call (e.g., ""Hello, my name is [Name], and I'm calling from [Insurance Company]. How can I assist you today?""). 3. **Verify customer information**: Confirm the customer's name, policy number, and other relevant details to ensure you're speaking with the correct person. **Call Handling** 1. **Listen actively**: Pay attention to the customer's concerns, ask clarifying questions, and paraphrase to ensure understanding. 2. **Empathize and show concern**: Acknowledge the customer's frustration or concern, and show empathy through phrases like ""I understand"" or ""I apologize for the inconvenience."" 3. **Provide clear explanations**: Clearly explain insurance policies, coverage, and procedures in a way that's easy for the customer to understand. 4. **Offer solutions and alternatives**: Provide options and solutions to address the customer's issue, and explain the next steps and timelines. 5. **Take ownership**: Take responsibility for resolving the customer's issue, and escalate the call to a supervisor or specialist if necessary. **Data Collection and Documentation** 1. **Gather relevant information**: Collect all necessary information to resolve the customer's issue, including policy numbers, claim numbers, and other relevant details. 2. **Document the call**: Accurately document the customer's issue, the resolution, and any follow-up actions required in the customer's file or CRM system. 3. **Update customer records**: Ensure that customer records are up-to-date and accurate, including contact information, policy details, and claim status. **Resolution and Follow-up** 1. **Resolve issues efficiently**: Strive to resolve the customer's issue on the first call, and provide a clear explanation of the resolution. 2. **Provide follow-up information**: Offer to follow up with the customer to ensure the issue is resolved to their satisfaction, and provide a timeline for follow-up. 3. **Thank the customer**: Express gratitude for the customer's business, and thank them for choosing the insurance company. **Additional Guidelines** 1. **Maintain confidentiality**: Ensure that all customer information is kept confidential, and only shared with authorized personnel. 2. **Be aware of regulatory requirements**: Familiarize yourself with relevant insurance regulations, such as HIPAA, and ensure compliance during calls. 3. **Use positive language**: Use positive and professional language throughout the call, avoiding jargon and technical terms that may confuse the customer. 4. **Escalate complex issues**: Know when to escalate complex issues to a supervisor or specialist, and do so in a timely manner to ensure the customer receives the necessary assistance. These guidelines can help insurance call center phone operators provide excellent customer service, resolve issues efficiently, and maintain a positive reputation for the insurance company."


In [0]:
drop table if exists call_center_guidelines;
create table if not exists call_center_guidelines (guidelines string);

insert into call_center_guidelines values (
"1. Inform customer that 'all calls are recorded for training and monitoring purposes'.
 2. Greet the customer with a friendly and professional tone.
 3. Verify customer information
 4. provide a clear explanation of the resolution.
 5. end the call by inform customer 'if you have more questions, you can always call us at 1-800-123-4567'"
);

num_affected_rows,num_inserted_rows
1,1


In [0]:
select 
*
from raw_transcripts limit 10

call_timestamp operator_id phone_number transcript _rescued_data 2024-07-18T10:33:55.000Z 7007 (819)-921-1828 My car's audio system was stolen and I need to file a claim., Oh, so you're another one of those people who thinks they can just leave their car unlocked and not have anything stolen, huh? Well, let me tell you, that's just not how the world works. You gotta take responsibility for your own property, pal. Now, I know you're here to file a claim for your precious audio system, but let me tell you, it's not gonna be easy. You're gonna have to jump through some hoops, provide all sorts of documentation, and maybe, just maybe, we'll cut you a check. But don't get your hopes up too high, pal. So, what kind of audio system was it? One of those fancy schmancy ones with all the bells and whistles? Ha! Like that's gonna make a difference. You're still gonna have to pay your deductible, and don't even get me started on the paperwork. But hey, I'm a nice guy, so I'll give you a break. I'll make sure to put a rush on your claim, even though it's not like you're gonna get a brand new audio system out of the deal. You'll be lucky if we give you a few bucks for a new radio. So, what do you say? Ready to dance with the devil and file that claim? Or are you gonna take your chances and hope your car doesn't get broken into again? The choice is yours, pal. But either way, don't come crying to me when things don't go your way., Sure, I'd be happy to help you file a claim for your stolen audio system. However, I must inform you that our agency's policy requires a thorough investigation before any claims can be approved. Additionally, our team of experts will need to review the circumstances surrounding the theft to ensure that all necessary protocols were followed. I understand that this process can be frustrating, but please rest assured that we are committed to providing the highest level of service to our clients. We will do everything in our power to make sure that your claim is processed as efficiently and effectively as possible. Regarding the audio system, I'm afraid that we cannot guarantee a brand-new replacement. However, we will do our best to provide a fair settlement based on the value of the stolen item. Please keep in mind that the settlement amount may be affected by factors such as depreciation and the age of the item. I hope this information helps clarify things for you. If you have any further questions or concerns, please don't hesitate to reach out. We're here to help in any way we can., I understand that you have a process to follow, but I'm concerned that it's going to take a long time to get my claim approved. Can you give me an idea of how long the investigation and review process typically takes? Also, I'm not sure if I have all the documentation that you need. Can you provide me with a list of what I'll need to provide in order to file the claim? I want to make sure that I have everything ready and in order to avoid any delays. Lastly, I'm not sure if I understand how the settlement amount is determined. Can you explain to me how you calculate the value of the stolen item and how depreciation affects the settlement amount? I appreciate your help and I'm willing to work with you to make sure that everything is in order, but I want to make sure that I'm well informed throughout the process., Sure, I'd be happy to help you file a claim for your stolen audio system. However, I must inform you that our agency's policy requires a thorough investigation before any claims can be approved. Additionally, our team of experts will need to review the circumstances surrounding the theft to ensure that all necessary protocols were followed. I understand that this process can be frustrating, but please rest assured that we are committed to providing the highest level of service to our clients. We will do everything in our power to make sure that your claim is processed as efficiently and effectively as possible. Regarding the au

In [0]:
select
  ai_query(
      "databricks-claude-3-7-sonnet",
      concat("You are an expert in customer relationship management, analysis the following transcript: ", "<transcript>", transcript, "</transcript>\n",
      "give a recommendation of the next best action from the following list of choices: ", 
      "'follow-up call', 'promotional email', 'automated email', 'apology email'",
      "Answer with one recommended action, do not explain. If no action is required, return 'none'."
      "Answer with a succinct justificaction of the recommendation in less than 5 words "),
      returnType => 'STRUCT<recommendation:STRUCT<catagory:STRING, justification:STRING>>',
      failOnError => false
    ) as next_best_action
from transcripts_ai_query_test limit 10

next_best_action
"List(follow-up call Claim needs personal attention, null)"
"List(follow-up call Verify claim status, null)"
"List(follow-up call Needs policy update confirmation, null)"
"List(follow-up call Confirm policy details needed, null)"
"List(apology email Extremely unprofessional customer service, null)"
"List(apology email (customer service was unprofessional), null)"
"List(follow-up call Confirm policy changes completed, null)"
"List(follow-up call Confirm coverage details needed, null)"
"List(apology email customer deserves formal apology, null)"
"List(apology email Unprofessional and dismissive service, null)"


In [0]:
SELECT
  ai_query(
      "qyu_llama33_ai_query",
      "extract the customer intent (of either 'auto accident', 'home accident', 'motocycle', or 'policy related') and
      key context of the intent from the transcript:" || transcript,
      responseFormat => 'STRUCT<intent_analysis:STRUCT<intent:STRING, context:STRING>>',
      failOnError => false
  ) as customer_intent,
  -- Next Best Action Recommendation
  ai_query(
      "qyu_llama33_ai_query",
      concat("You are an expert in customer relationship management, analysis the following transcript: ", "<transcript>", transcript, "</transcript>\n",
      "give a recommendation of the next best action from the following list of choices: ", 
      "'follow-up call', 'promotional email', 'automated email', 'apology email'",
      "Answer with one recommended action, do not explain. If no action is required, return 'none'."
      "Answer with a succinct justificaction of the recommendation in less than 5 words "),
      responseFormat => 'STRUCT<recommendation:STRUCT<catagory:STRING, justification:STRING>>'
    ) as next_best_action,
    
  -- Customer asks
  ai_query(
     "databricks-claude-3-7-sonnet",
     concat("Analysis the following transcripts: ", "<transcript>", transcript, "<transcript>\n",
          "identify the main question and concerns from the customer in one sentence")
  ) AS customer_asks,
  transcript
FROM fins_genai.call_center.call_center_transcripts_cleaned
limit 10

customer_intent next_best_action customer_asks transcript Map(result -> {"intent": "policy related", "context": "add a valuable item to homeowner"}, errorMessage -> null) {"catagory": "none", "justification": "Issue resolved"} The main question from the customer is to add a valuable painting worth $50,000 that they recently inherited to their homeowner's insurance policy to ensure it's properly protected. all calls are recorded for training and monitoring purposes. I want to add a valuable item to my homeowner's insurance policy., Hello, thank you for contacting us today. I see from your policy information that you have policy number 123456 with us for your home insurance. Regarding your request to add a valuable item to your policy, I'll be happy to assist you with that. Could you please provide me with more details about the item you wish to add? Specifically, I need the following information: A detailed description of the item, including any unique features or characteristics. The appraised value of the item. The date of purchase or appraisal. Once you provide me with this information, I can help you add the item to your policy and ensure that it is properly covered against any potential losses or damages. If you have any questions or concerns about the process, please let me know and I'll be happy to help., Hello, thank you for your assistance. The item I would like to add to my policy is a rare and valuable painting that I recently inherited from a relative. Here are the details you requested: The painting is an original work by a well-known artist, and it is in excellent condition. It measures approximately 30 inches by 40 inches and features a beautiful landscape scene. The painting is framed in a decorative gold frame, which adds to its value. I had the painting appraised by a professional appraiser, and its estimated value is $50, The appraisal was conducted last month, and I have the official appraisal document on hand. I hope this information is sufficient for you to add the painting to my policy. Please let me know if there is any additional information you need from me. I want to ensure that this valuable item is properly protected., Thank you for providing me with the details of your valuable painting. I have updated your policy information accordingly, and the painting is now covered under your homeowner's insurance policy with a value of $50, I would like to remind you that it is important to keep the appraisal document in a safe place, as it will be necessary in the event of a claim. Additionally, if the value of the painting increases over time, it is recommended that you have it reappraised and update your policy accordingly to ensure that it remains fully covered. If you have any further questions or concerns about your policy or the coverage for your valuable painting, please don't hesitate to contact us. We are here to help you protect what matters most. Once again, thank you for choosing our insurance company for your homeowner's insurance needs. We appreciate your business and look forward to continuing to serve you., Thank you for the update and for adding the painting to my policy. I appreciate your help in ensuring that this valuable item is properly protected. I will make sure to keep the appraisal document in a safe place, as you recommended. I will also keep in mind the importance of updating the appraisal and my policy if the value of the painting increases over time. I am very satisfied with the level of service I have received from your company. Your team has been very helpful and responsive to my needs, and I feel confident that my home and valuable items are well-protected. Once again, thank you for your assistance and for the excellent service you have provided. If I have any further questions or concerns, I will not hesitate to contact you. Have a great day!, You're welcome! I'm glad I could help you add your valuable painting to your homeowner's insurance policy. Keeping the appraisal docu

In [0]:
select
  sum(unprofessionalism),
  sum(no_detail_explaination),
  sum(no_identity_verification),
  sum(no_inform_compliance),
  sum(no_inform_company_contact)
from fins_genai.call_center.call_center_transcripts_analysis

sum(unprofessionalism),sum(no_detail_explaination),sum(no_identity_verification),sum(no_inform_compliance),sum(no_inform_company_contact)
328,164,352,36,107
